In [1]:
import mapping_tools as mt
import numpy as np
import sys
import os
import gzip
import re
import pandas as pd
from timeit import default_timer as timer
from VEPcrossref import VEPfileCrossrefGenerator as cr

In [ ]:
# INTERFACES PART

In [14]:
# Extract the info corresponding to the prot ID (Interface parse)

#sys.argv[1]
#sys.argv[2]

protID = 'ENSP00000482258'
int_db_file = '/home/vruizser/PhD/2018-2019/Immunity_interfaces_analysis/raw_data/interfaces_mapped_to_v94.csv'

  
interfacesDB = open(int_db_file, "r")
cols = interfacesDB.readline().strip().split(" ")
#prot_interface = mt.ProtIntDB_parser(interfacesDB, protID)
prot_interface = mt.parser(interfacesDB,  protID, cols, " " )

In [56]:
subset_prot_interface = prot_interface[['pdb.id',
                      'ensembl.prot.id',
                      'temp.chain',
                      'int.chain',
                      'interaction',
                      'resid_sseq',
                      'mapped.real.pos',
                      'pdb.pos' ]]

# put it into right format
subset_prot_interface.columns = subset_prot_interface.columns.str.replace("\\.", "_")

for col in ("resid_sseq", "mapped_real_pos", "pdb_pos"):
    subset_prot_interface.loc[ : , col] = subset_prot_interface[col].str.replace("-", ',')
    subset_prot_interface.loc[ : , col] = subset_prot_interface[col].str.split(',')

subset_prot_interface = mt.explode(subset_prot_interface, ["resid_sseq", "mapped_real_pos", "pdb_pos"])

In [ ]:
####################### VARIANTS PART #####################################################

In [57]:
# Translate protID to gene ID
subset_prot_interface

,pdb_id,ensembl_prot_id,temp_chain,int_chain,interaction,resid_sseq,mapped_real_pos,pdb_pos
0,1gzh.pdb1,ENSP00000482258,A,B,protein,Y,163,69
1,1gzh.pdb1,ENSP00000482258,A,B,protein,Q,165,71
2,1gzh.pdb1,ENSP00000482258,A,B,protein,Q,167,73
3,1gzh.pdb1,ENSP00000482258,A,B,protein,Q,167,73
4,1gzh.pdb1,ENSP00000482258,A,B,protein,Q,167,73
5,1gzh.pdb1,ENSP00000482258,A,B,protein,H,168,74
6,1gzh.pdb1,ENSP00000482258,A,B,protein,H,168,74
7,1gzh.pdb1,ENSP00000482258,A,B,protein,H,168,74
8,1gzh.pdb1,ENSP00000482258,A,B,protein,E,171,77
9,1gzh.pdb1,ENSP00000482258,A,B,protein,E,171,77


In [20]:
biomartdb = open("/home/vruizser/PhD/2018-2019/git/PDBmapper/project/gene_transcript_protein_ens_ids.txt", "r")
ensemblIDs = mt.ensemblID_translator(biomartdb, protID)
ensemblIDs["geneID"]

'ENSG00000141510'

In [21]:
# get the corresponding VEP file
crossref_file = open("/home/vruizser/PhD/2018-2019/git/PDBmapper/project/geneids_VEPfiles_crossref.txt", "r")
VEP_filename = mt.VEP_getter(crossref_file, ensemblIDs["geneID"])

In [22]:
VEP_dir= "/home/vruizser/PhD/2018-2019/PanCancer_data/vep_output/"
VEPfile = open(VEP_dir + VEP_filename, "r")
geneID = ensemblIDs['geneID']
cols = pd.read_csv(VEPfile, nrows = 0, skiprows = 42, sep = "\t").columns
VEP = mt.parser(VEPfile,  geneID, cols, "\t" )

In [73]:

def MapVariantToPDB(variantDF, interfacesDF):

#     # Merge them both files
    interfacesDF.rename(columns={'mapped_real_pos':'Protein_position'}, inplace=True)
    df = pd.merge(variantDF, interfacesDF,on=["Protein_position"],how='inner')

#     # Save the merged dataframe
#     df.to_csv('merged.csv', index=False)
    return df

bien = MapVariantToPDB( VEP, subset_prot_interface)
bien.columns

Index(['#Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature',
       'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position',
       'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation',
       'Extra', 'pdb_id', 'ensembl_prot_id', 'temp_chain', 'int_chain',
       'interaction', 'resid_sseq', 'pdb_pos'],
      dtype='object')

In [82]:
bien['Allele']

0        G
1        G
2        G
3        G
4        G
5        G
6        G
7        G
8        G
9        G
10       G
11       G
12       G
13       G
14       G
15       G
16       G
17       G
18       G
19       G
20       G
21       G
22       G
23       G
24       G
25       G
26       G
27       G
28       G
29       G
        ..
33893    T
33894    T
33895    T
33896    T
33897    T
33898    T
33899    T
33900    T
33901    T
33902    T
33903    T
33904    T
33905    T
33906    T
33907    T
33908    T
33909    T
33910    T
33911    T
33912    T
33913    T
33914    T
33915    T
33916    T
33917    -
33918    -
33919    -
33920    -
33921    -
33922    -
Name: Allele, Length: 33923, dtype: object

In [72]:
# define main function to execute the previous defined functions together
def main():

 

    
    
    return


##########################
# execute main function  #
##########################
if __name__ == '__main__':
    main()